In [1]:
!git clone https://github.com/CASIA-IVA-Lab/FastSAM.git
%pip  install -r FastSAM/requirements.txt
# install CLIP
%pip  install git+https://github.com/openai/CLIP.git
# install SAM
%pip  install git+https://github.com/facebookresearch/segment-anything.git
# install other dependencies
%pip  install roboflow supervision jupyter_bbox_widget
%pip install roboflow

Cloning into 'FastSAM'...
remote: Enumerating objects: 1171, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 1171 (delta 201), reused 186 (delta 186), pack-reused 929
Receiving objects: 100% (1171/1171), 72.45 MiB | 33.54 MiB/s, done.
Resolving deltas: 100% (456/456), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.7/611.7 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="OZmt7veKofDCRjgyQ5XE")
project = rf.workspace("csilab").project("csilab-braintumor")
dataset = project.version(3).download("yolov8")

import os
HOME = os.getcwd()
print("HOME:", HOME)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.120, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to CsiLab-BrainTumor-3 in yolov8:: 100%|██████████| 618/618 [00:00<00:00, 10615.01it/s]

HOME: /content


In [4]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt


In [5]:
FAST_SAM_CHECKPOINT_PATH = "/content/weights/FastSAM.pt"

In [6]:
%cd {HOME}/FastSAM

import os
import cv2
import torch
import roboflow
import base64

import supervision as sv
import numpy as np

from roboflow import Roboflow
from fastsam import FastSAM, FastSAMPrompt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

/content/FastSAM


In [7]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

fast_sam = FastSAM(FAST_SAM_CHECKPOINT_PATH)

In [90]:
def convert_masks_to_yolo_labels(masks):

  labels = []
  ob_id = 0
  for mask, in zip(masks):
    # Move the mask tensor to the CPU
    mask_cpu = mask.cpu()

    # Convert the mask tensor to a NumPy array
    mask_np = mask_cpu.numpy()

    bounding_box = cv2.boundingRect(mask_np.astype(np.uint8))

    x_center = bounding_box[0] + bounding_box[2] // 2
    y_center = bounding_box[1] + bounding_box[3] // 2
    width = bounding_box[2]
    height = bounding_box[3]

    x_center = x_center / mask.shape[1]
    y_center = y_center / mask.shape[0]
    width = width / mask.shape[1]
    height = height / mask.shape[0]

    label = [ob_id, x_center, y_center, width, height]
    labels.append(label)
    ob_id+=1


  return labels

In [ ]:
import os
path = dataset.location + "/train/images/"
Output_Path = dataset.location +"/output/"

if not os.path.exists(Output_Path):
    os.makedirs(Output_Path)
    print("Criado diretorio: ", Output_Path)

for IMAGE in os.listdir(path):
   if (IMAGE.endswith(".jpg")):
      IMAGE_PATH = path+"/"+IMAGE
      results = fast_sam(
        source=IMAGE_PATH,
        device=DEVICE,
        retina_masks=True,
        imgsz=1024,
        conf=0.5,
        iou=0.5)
      prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
      masks = prompt_process.everything_prompt()
      prompt_process.plot(annotations=masks, output_path = Output_Path + IMAGE)
      #torch.save(masks, (Output_Path + IMAGE).split(".")[0] + ".pt")
      yolo_labels = convert_masks_to_yolo_labels(masks)

      '''out_file_path = (Output_Path + IMAGE).split(".jpg")[0] + ".txt"
      with open(out_file_path, "w") as out_file:
          for label in yolo_labels:
              out_file.write(" ".join(map(str, label)) + "\n")'''
      #separando por objetos detectatos
      for label in yolo_labels:
        out_file_path = (Output_Path + IMAGE).split(".jpg")[0]+ "object_" + str(label[0]) + ".txt"
        out_file = open(out_file_path, "w")
        out_file.write(" ".join(map(str, label)) + "\n")



In [71]:
yolo_labels = convert_masks_to_yolo_labels(masks)
for label in yolo_labels:
  print(label[0])



#model = torch.load('/content/CsiLab-BrainTumor-3/output/45_jpg.pt')


0
1


In [ ]:
IMAGE_PATH = os.path.join(dataset.location, 'train', '45_jpg.rf.36ef270424c1c2114e972a731368c5df.jpg')

In [91]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=False,
    imgsz=1024,
    conf=0.5,
    iou=0.5)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
masks = prompt_process.everything_prompt()
prompt_process.plot(annotations=masks, output_path = f"{HOME}/output" + ".jpg")

yolo_labels = convert_masks_to_yolo_labels(masks)
print(yolo_labels)



image 1/1 /content/CsiLab-BrainTumor-3/train/images/26_jpg.rf.836b5a9a4de03c03ac33fbc38e8002da.jpg: 1024x1024 2 objects, 165.4ms
Speed: 5.8ms preprocess, 165.4ms inference, 4.0ms postprocess per image at shape (1, 3, 1024, 1024)


[[0, 0.4072265625, 0.3974609375, 0.166015625, 0.208984375], [1, 0.517578125, 0.49609375, 0.8544921875, 0.8828125]]


In [74]:
def annotate_image(image_path: str, masks: np.ndarray) -> np.ndarray:
    image = cv2.imread(image_path)

    xyxy = sv.mask_to_xyxy(masks=masks)
    detections = sv.Detections(xyxy=xyxy, mask=masks)

    mask_annotator = sv.MaskAnnotator()
    return mask_annotator.annotate(scene=image.copy(), detections=detections)

In [75]:
def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [76]:
IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget

widget = BBoxWidget()
widget.image = encode_image(IMAGE_PATH)
widget

BBoxWidget(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#b…

In [77]:

widget.bboxes[0]['label'] = "tumor"
widget.bboxes

[{'x': 70, 'y': 63, 'width': 43, 'height': 48, 'label': 'tumor'}]

In [78]:

box = widget.bboxes[0] if widget.bboxes else default_box
box = [
    box['x'],
    box['y'],
    box['x'] + box['width'],
    box['y'] + box['height']
]
print(box)

[70, 63, 113, 111]


In [98]:
def convert_masks_to_yolo_labels(masks):

  labels = []
  ob_id = 0
  for mask, in zip(masks):
    # Move the mask tensor to the CPU
    #mask_cpu = mask.cpu()

    # Convert the mask tensor to a NumPy array
    #mask_np = mask_cpu.numpy()

    bounding_box = cv2.boundingRect(mask.astype(np.uint8))

    x_center = bounding_box[0] + bounding_box[2] // 2
    y_center = bounding_box[1] + bounding_box[3] // 2
    width = bounding_box[2]
    height = bounding_box[3]

    x_center = x_center / mask.shape[1]
    y_center = y_center / mask.shape[0]
    width = width / mask.shape[1]
    height = height / mask.shape[0]

    label = [ob_id, x_center, y_center, width, height]
    labels.append(label)
    ob_id+=1


  return labels

In [106]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=True,
    imgsz=1024,
    conf=0.5,
    iou=0.5)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
prompt_process.plot(annotations=masks, output_path = f"{HOME}/output" + ".jpg")
masks = prompt_process.box_prompt(bbox=box)

yolo_labels = convert_masks_to_yolo_labels(masks)

for label in yolo_labels:
  out_file_path = f"{HOME}/output_"+ "object_" + str(label[0]) + ".txt"
  out_file = open(out_file_path, "w")
  out_file.write(" ".join(map(str, label)) + "\n")
  print(label)

#converter mask to .txt to yollov8
#criear dataset com imagens instanciadas no roboflow


image 1/1 /content/CsiLab-BrainTumor-3/train/images/26_jpg.rf.836b5a9a4de03c03ac33fbc38e8002da.jpg: 1024x1024 2 objects, 173.4ms
Speed: 5.8ms preprocess, 173.4ms inference, 4.2ms postprocess per image at shape (1, 3, 1024, 1024)


[0, 0.40625, 0.39732142857142855, 0.16964285714285715, 0.20535714285714285]


In [94]:
def convert_masks_to_yolo_labels(masks, class_ids):
  labels = []
  for mask, class_id in zip(masks, class_ids):
    bounding_box = cv2.boundingRect(mask.astype(np.uint8))
    x_center = bounding_box[0] + bounding_box[2] // 2
    y_center = bounding_box[1] + bounding_box[3] // 2
    width = bounding_box[2]
    height = bounding_box[3]

    x_center = x_center / mask.shape[1]
    y_center = y_center / mask.shape[0]
    width = width / mask.shape[1]
    height = height / mask.shape[0]

    label = [class_id, x_center, y_center, width, height]
    labels.append(label)

  return labels

print(convert_masks_to_yolo_labels(masks, [1])[0])

[1, 0.40625, 0.39732142857142855, 0.16964285714285715, 0.20535714285714285]


In [86]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=True,
    imgsz=1024,
    conf=0.5,
    iou=0.9)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
masks = prompt_process.everything_prompt()


image 1/1 /content/CsiLab-BrainTumor-3/train/images/26_jpg.rf.836b5a9a4de03c03ac33fbc38e8002da.jpg: 1024x1024 3 objects, 163.1ms
Speed: 6.9ms preprocess, 163.1ms inference, 4.5ms postprocess per image at shape (1, 3, 1024, 1024)


In [87]:
masks = masks.cpu().numpy().astype(bool)

annotated_image=annotate_image(image_path=IMAGE_PATH, masks=masks)
sv.plot_image(image=annotated_image, size=(8, 8))

ValueError: ignored

In [83]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=False,
    imgsz=1024,
    conf=0.5,
    iou=0.9)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
masks = prompt_process.text_prompt(text='lighter brain tumor inside an brain')
prompt_process.plot(annotations=masks, output_path = f"{HOME}/output" + ".jpg")


image 1/1 /content/CsiLab-BrainTumor-3/train/images/26_jpg.rf.836b5a9a4de03c03ac33fbc38e8002da.jpg: 1024x1024 3 objects, 173.3ms
Speed: 5.7ms preprocess, 173.3ms inference, 3.2ms postprocess per image at shape (1, 3, 1024, 1024)
100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 51.5MiB/s]


In [84]:
annotated_image=annotate_image(image_path=IMAGE_PATH, masks=masks)
sv.plot_image(image=annotated_image, size=(8, 8))

ValueError: ignored